In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder
    .appName("DeltaLakeS3")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .getOrCreate()

In [ ]:
from pyspark.sql.functions import count, max

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [ ]:
from delta.tables import DeltaTable

In [ ]:
delta_table_path = "s3a://qd-transaction-ambking-001/table/bronze"

In [ ]:
deltaTable = DeltaTable.forPath(spark, delta_table_path)

In [ ]:
delta_table_path_sl_com = "s3a://qd-gaming-uc-v2/silver/silver_precal/tables/77e725e3-9c0a-fsdfdhs4c28-93c8-1da76b000674"

In [ ]:
df = spark.read.format("delta").load(delta_table_path_sl_com)

In [ ]:
df.printSchema()

In [ ]:
result_df = (
    df.groupBy("pre_cal_id")
      .agg(
          count("*").alias("record_count"),
          max("update_date_ts").alias("latest_update_date_ts")
      )
      .orderBy("latest_update_date_ts", ascending=False)
)

result_df.show(truncate=False)